In [336]:
print("hello")

hello


In [337]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestRegressor


In [338]:
dataset=pd.read_csv("train.csv")

In [339]:
dataset.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [340]:
dataset.shape

(188533, 13)

In [341]:
dataset.describe()

,id,model_year,milage,price
count,188533.000000,188533.000000,188533.000000,1.885330e+05
mean,94266.000000,2015.829998,65705.295174,4.387802e+04
std,54424.933488,5.660967,49798.158076,7.881952e+04
min,0.000000,1974.000000,100.000000,2.000000e+03
25%,47133.000000,2013.000000,24115.000000,1.700000e+04
50%,94266.000000,2017.000000,57785.000000,3.082500e+04
75%,141399.000000,2020.000000,95400.000000,4.990000e+04
max,188532.000000,2024.000000,405000.000000,2.954083e+06


In [342]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [343]:
dataset.isnull().sum()

id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

In [344]:
dataset.drop(columns=["id","int_col","clean_title"], axis=1, inplace=True)

In [345]:
dataset.shape

(188533, 10)

In [346]:
dataset.dropna(inplace=True)

In [347]:
dataset.isnull().sum()

brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
accident        0
price           0
dtype: int64

In [348]:
dataset.shape

(181067, 10)

In [349]:
dataset["fuel_type"].value_counts()

fuel_type
Gasoline          163853
Hybrid              6741
E85 Flex Fuel       5328
Diesel              3854
–                    764
Plug-In Hybrid       512
not supported         15
Name: count, dtype: int64

In [350]:
dataset=dataset[dataset["fuel_type"]!="-"]

In [351]:
dataset["fuel_type"].value_counts()

fuel_type
Gasoline          163853
Hybrid              6741
E85 Flex Fuel       5328
Diesel              3854
–                    764
Plug-In Hybrid       512
not supported         15
Name: count, dtype: int64

In [352]:
# Drop rows where 'fuel_type' is '-'
dataset = dataset[dataset["fuel_type"] != "–"]  # Note that it's a long dash '–' (not a regular hyphen '-')

# Alternatively using .loc:
# dataset = dataset.loc[dataset["fuel_type"] != "–"]


In [353]:
dataset["fuel_type"].value_counts()

fuel_type
Gasoline          163853
Hybrid              6741
E85 Flex Fuel       5328
Diesel              3854
Plug-In Hybrid       512
not supported         15
Name: count, dtype: int64

In [354]:
dataset.shape

(180303, 10)

In [355]:
dataset.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,accident,price
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,None reported,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,At least 1 accident or damage reported,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,None reported,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,None reported,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,None reported,97500


In [356]:
dataset["brand"].value_counts()

brand
Ford             22238
Mercedes-Benz    18929
BMW              16768
Chevrolet        16036
Audi             10667
Porsche          10169
Land              9417
Toyota            8674
Lexus             8573
Jeep              6281
Cadillac          4569
RAM               4169
Nissan            3802
INFINITI          3235
GMC               3138
Dodge             2878
Mazda             2635
Kia               2396
Lincoln           2388
Subaru            2369
Acura             2236
Honda             2047
Hyundai           1980
Volkswagen        1715
Jaguar            1282
Bentley           1122
MINI              1049
Genesis            951
Buick              927
Maserati           926
Lamborghini        792
Volvo              695
Chrysler           684
Alfa               664
Rolls-Royce        550
Mitsubishi         542
Pontiac            532
Hummer             516
Tesla              371
Ferrari            349
McLaren            241
Aston              235
Saturn             145
Scion

In [357]:
dataset["ext_col"].value_counts()

ext_col
Black                    46884
White                    41899
Gray                     24033
Silver                   16283
Blue                     13849
                         ...  
Dark Gray Metallic           5
Frozen White                 5
Thunder Gray                 4
BLUE                         2
Mango Tango Pearlcoat        1
Name: count, Length: 319, dtype: int64

In [358]:
dataset["engine"].value_counts()

engine
355.0HP 5.3L 8 Cylinder Engine Gasoline Fuel           3451
240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel           2876
420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel           2805
2.0L I4 16V GDI DOHC Turbo                             2633
375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel          2430
                                                       ... 
500.0HP Electric Motor Electric Fuel System               1
443.0HP 4.4L 8 Cylinder Engine Gasoline Fuel              1
139.0HP 1.6L 4 Cylinder Engine Gas/Electric Hybrid        1
533.0HP Electric Motor Electric Fuel System               1
313.0HP 2.0L 4 Cylinder Engine Plug-In Electric/Gas       1
Name: count, Length: 1108, dtype: int64

In [359]:
dataset["accident"].value_counts()

accident
None reported                             139241
At least 1 accident or damage reported     41062
Name: count, dtype: int64

In [360]:
dataset['accident'].replace({"None reported": "0","At least 1 accident or damage reported": "1"}, inplace=True)

In [361]:
dataset.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,accident,price
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,0,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,1,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,0,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,0,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,0,97500


In [362]:
dataset["transmission"].value_counts()

transmission
A/T                                                   46617
8-Speed A/T                                           20248
Transmission w/Dual Shift Mode                        18981
6-Speed A/T                                           17854
6-Speed M/T                                           11751
7-Speed A/T                                           10967
Automatic                                             10170
8-Speed Automatic                                      8175
10-Speed A/T                                           7915
9-Speed A/T                                            3806
5-Speed A/T                                            3149
10-Speed Automatic                                     3037
6-Speed Automatic                                      2704
4-Speed A/T                                            2460
5-Speed M/T                                            2317
9-Speed Automatic                                      2276
CVT Transmission           

In [363]:
Lb=LabelEncoder()

In [364]:
dataset["brand"]=Lb.fit_transform(dataset["brand"])
dataset["model"]=Lb.fit_transform(dataset["model"])
dataset["fuel_type"]=Lb.fit_transform(dataset["fuel_type"])
dataset["transmission"]=Lb.fit_transform(dataset["transmission"])
dataset["ext_col"]=Lb.fit_transform(dataset["ext_col"])
dataset["engine"]=Lb.fit_transform(dataset["engine"])


In [365]:
dataset.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,accident,price
0,31,493,2007,213000,2,112,38,312,0,4200
1,28,926,2002,143250,2,360,38,263,1,4999
2,9,1569,2002,136731,1,633,38,38,0,13900
3,16,755,2017,19500,2,855,49,29,0,45000
4,36,1073,2021,7388,2,254,23,29,0,97500


In [366]:
dataset.describe()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,price
count,180303.000000,180303.000000,180303.000000,180303.000000,180303.000000,180303.000000,180303.000000,180303.000000,1.803030e+05
mean,22.585592,856.770448,2015.732694,66794.954260,1.971015,595.196602,29.876464,157.124679,4.329056e+04
std,15.509465,523.715563,5.596915,49849.996723,0.404606,271.768805,12.766217,114.706075,7.840468e+04
min,0.000000,0.000000,1974.000000,100.000000,0.000000,0.000000,0.000000,0.000000,2.000000e+03
25%,9.000000,421.000000,2013.000000,25000.000000,2.000000,371.000000,20.000000,29.000000,1.680000e+04
50%,20.000000,792.000000,2017.000000,59300.000000,2.000000,605.000000,32.000000,128.000000,3.000000e+04
75%,36.000000,1317.000000,2020.000000,96473.000000,2.000000,820.000000,38.000000,281.000000,4.881500e+04
max,56.000000,1886.000000,2024.000000,405000.000000,5.000000,1107.000000,51.000000,318.000000,2.954083e+06


In [367]:
X=dataset.drop("price",axis=1)
Y=dataset["price"]

In [368]:
Y.head()

0     4200
1     4999
2    13900
3    45000
4    97500
Name: price, dtype: int64

In [369]:
regresoo=RandomForestRegressor(       # Increase or decrease this based on dataset size
    max_depth=10,
    max_features='sqrt',  # Limit the number of features at each split
    random_state=42)

In [370]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [371]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((144242, 9), (36061, 9), (144242,), (36061,))

In [372]:
regresoo.fit(X_train,Y_train)

RandomForestRegressor(max_depth=10, max_features='sqrt', random_state=42)

In [373]:
X_train_pred=regresoo.predict(X_train)
r2_score(X_train_pred,Y_train)

-2.4540472148882966

In [374]:
X_test_pred=regresoo.predict(X_test)
r2_score(X_test_pred,Y_test)

-6.119231247581154

In [375]:
scores = cross_val_score(regresoo, X_train, Y_train, cv=5, scoring='r2')
print("Cross-validation R² scores:", scores)
print("Average R² score:", scores.mean())

Cross-validation R² scores: [0.14161794 0.128027   0.12420433 0.1503252  0.17361033]
Average R² score: 0.14355696189653905


In [379]:
result=regresoo.score(X_train,Y_train)
result1=regresoo.score(X_test,Y_test)
result
result1

0.13699881039843886

In [380]:
X.shape

(180303, 9)

In [381]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import randint, uniform

# Load the dataset
dataset = pd.read_csv("train.csv")

# Data Preprocessing
dataset.drop(columns=["id", "int_col", "clean_title"], axis=1, inplace=True)
dataset.dropna(inplace=True)

# Drop rows with invalid fuel types
dataset = dataset[dataset["fuel_type"].isin(["-", "–"]) == False]

# Replace accident labels with binary values
dataset['accident'].replace({"None reported": 0, "At least 1 accident or damage reported": 1}, inplace=True)

# Encode categorical variables
label_cols = ["brand", "model", "fuel_type", "transmission", "ext_col", "engine"]
Lb = LabelEncoder()
for col in label_cols:
    dataset[col] = Lb.fit_transform(dataset[col])

# Define features and target variable
X = dataset.drop("price", axis=1)
Y = dataset["price"]

# Train-Test Split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Hyperparameter tuning using RandomizedSearchCV
param_dist = {
    'n_estimators': [100, 200],
    'learning_rate': uniform(0.01, 0.1),  # Random float from 0.01 to 0.1
    'max_depth': randint(3, 8),  # Random integer from 3 to 7
    'min_samples_split': randint(2, 10),  # Random integer from 2 to 9
    'min_samples_leaf': randint(1, 5)  # Random integer from 1 to 4
}

# Initialize GradientBoostingRegressor
model = GradientBoostingRegressor(random_state=42)

# Perform RandomizedSearchCV to find the best hyperparameters
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, 
                                   n_iter=50, cv=5, scoring='r2', verbose=2, random_state=42, n_jobs=-1)
random_search.fit(X_train, Y_train)

# Best parameters from Randomized Search
print("Best parameters found: ", random_search.best_params_)

# Train the model with the best parameters
best_model = random_search.best_estimator_
best_model.fit(X_train, Y_train)

# Predict and evaluate
Y_train_pred = best_model.predict(X_train)
Y_test_pred = best_model.predict(X_test)

# Evaluate the model
print("Training R² score:", r2_score(Y_train, Y_train_pred))
print("Testing R² score:", r2_score(Y_test, Y_test_pred))
print("Testing MSE:", mean_squared_error(Y_test, Y_test_pred))

# Optional: Cross-validation scores
from sklearn.model_selection import cross_val_score

scores = cross_val_score(best_model, X_train, Y_train, cv=5, scoring='r2')
print("Cross-validation R² scores:", scores)
print("Average R² score:", scores.mean())

# Optional: Feature Importance
importances = best_model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found:  {'learning_rate': 0.09957635956735193, 'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 6, 'n_estimators': 200}
Training R² score: 0.17935567954115694
Testing R² score: 0.1404142350837313
Testing MSE: 6132954716.336365
Cross-validation R² scores: [0.14965566 0.13451327 0.13078044 0.16226017 0.18407128]
Average R² score: 0.15225616249921547
        Feature  Importance
3        milage    0.572201
5        engine    0.188879
2    model_year    0.105881
1         model    0.060643
0         brand    0.035973
7       ext_col    0.018276
6  transmission    0.012820
4     fuel_type    0.003426
8      accident    0.001900
